# Continue Pretraining on T5

In [1]:
from datasets import load_dataset

tokenized_dataset = load_dataset('billingsmoore/temp2',split='train', streaming=True)

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('../../Models/my_tokenizer')

## Train the Model

In [3]:
%env WANDB_PROJECT=english-v2

env: WANDB_PROJECT=english-v2


In [4]:
from transformers import TrainingArguments, Trainer, T5ForConditionalGeneration, Adafactor
from accelerate import Accelerator

model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-large", device_map='cuda:0')
model.resize_token_embeddings(len(tokenizer))

accelerator = Accelerator()

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=False, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

model, optimizer = accelerator.prepare(model, optimizer)

training_args = TrainingArguments(
    output_dir="buddhist-large-pretrain",
    auto_find_batch_size=True,
    learning_rate=3e-4,
    max_steps=860_000,
    logging_steps=775,
    save_steps=10_000,
    save_safetensors=True
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    optimizers=(optimizer, None)
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
775,0.348100
1550,0.241400
2325,0.216800
3100,0.216300
3875,0.205400
4650,0.202400
5425,0.206100
6200,0.207300
6975,0.195000
7750,0.201300


In [6]:
model.save_pretrained('../Models/pretrained-large-model')